## Worked example: Counting Email in a Database

We used data provided by Dr. Chuck in his webpage. 

Unfortunately there is not syntax highlights for SQL in Jupyter notebooks

In [1]:
#Import sqlite library
import sqlite3

In [2]:
#we make the conection, and it'll create the emaildb file
conn = sqlite3.connect('emaildb.sqlite')

#the cursor is like a handle. You send SQL commands through the cursor
#and get responses through the same cursor
cur = conn.cursor() 

In [3]:
# this lines drops the existing table if exists, to start a fresh DB
cur.execute('DROP TABLE IF EXISTS Counts')

#Then we create a table that has 2 columns, email and count
cur.execute('''
CREATE TABLE Counts (email TEXT, count INTEGER)''')

In [4]:
fname = input('Enter file name: ')

if (len(fname) < 1): fname = 'data/mbox-short.txt' #set a default
fh = open(fname)

for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1]
    
    #we put ? because put exact strings, we want to prevent SQL injection
    #The question mark is a place holder, what prevents injection.
    #It needs a tuple, that is why the syntax is (email,)
    cur.execute('SELECT count FROM Counts WHERE email = ? ', (email,))
    #grab the first one and give it back into `row`
    row = cur.fetchone()
    
    if row is None:
        #if row is not there, insert it and count 1
        cur.execute('''INSERT INTO Counts (email, count)
                VALUES (?, 1)''', (email,))
    else:
        #if row existis update the value, update is safer than over 
        #writing since there can be other apps talking to the DB
        cur.execute('UPDATE Counts SET count = count + 1 WHERE email = ?',
                    (email,))
    #Here is when we write to disk. Now is every time but you can 
    #commit every a 10 records or something
    conn.commit()

Enter file name: data/mbox.txt


In [5]:
# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT email, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close() #close the conection

zqian@umich.edu 195
mmmay@indiana.edu 161
cwen@iupui.edu 158
chmaurer@iupui.edu 111
aaronz@vt.edu 110
ian@caret.cam.ac.uk 96
jimeng@umich.edu 93
rjlowe@iupui.edu 90
dlhaines@umich.edu 84
david.horwitz@uct.ac.za 67


**Now we can open the data base on sqlite browser. We can run the code again but now with the full file `mbox.txt` and we will see that it'll drop the table and if we hit refresh the new one will be there**
